## **DhakaAI Inference Code**

***Download YOLOv5 repository***

In [ ]:
%cd /content/
!git clone https://github.com/Morshed-Alam/yolov5.git  # clone repo
!pip install -qr yolov5/requirements.txt  # install dependencies (ignore errors)
%cd yolov5
 
import torch
from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets

!rm -r /content/sample_data/
clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

***Model download***

In [ ]:
%cd /content/
!gdown --id 10B4Za9wJm1PDhvkqzomrCT4kmLpg_9AT

***Test data download***

> Change test set here



In [ ]:
%cd /content/
!gdown --id 152cROdBccTeFv4WxBThByla3hLblY4qW
!unzip test2.zip; rm test2.zip;

***Inference***

In [ ]:
# use the best weights!
%cd /content/yolov5/
!python detect.py --weights ../best.pt \
                  --img 1024 \
                  --save-txt \
                  --augment \
                  --conf 0.6 \
                  --save-conf \
                  --iou-thres 0.5 \

                  --source ../test/ \
                  # ======== change test set here =========

***Create submission file***

> Generated csv file in /content/





In [ ]:
%cd /content/yolov5/inference/

import pandas as pd
import numpy as np
import os
import shutil

classes = ['ambulance','army vehicle','auto rickshaw','bicycle','bus','car','garbagevan','human hauler','minibus','minivan','motorbike','pickup','policecar','rickshaw','scooter','suv','taxi','three wheelers (CNG)','truck','van','wheelbarrow']


os.mkdir('output/images')
os.mkdir('output/labels')

fileNames = os.listdir('output/')
for file in fileNames:
    if file[-3:]=='jpg':
        shutil.move("output/"+file, "output/images/"+file)
    elif file[-3:]=='txt':
        shutil.move("output/"+file, "output/labels/"+file)
    else:
        continue

textLabels = os.listdir('output/labels')

df = pd.DataFrame(columns = ['image_id', 'class','score','xmin','ymin','xmax','ymax','width','height'])
for oneImage in textLabels:
    subdf = pd.read_csv("output/labels/"+oneImage, header = None,delim_whitespace=True)
    subdf['imageID'] = oneImage[:-4]
    for i, eachObject in subdf.iterrows():
        conf = round(float(eachObject[5]),2)
        new_row = {'image_id': eachObject['imageID'], 'class':classes[eachObject[0]],'score':conf,'xmin':eachObject[1],'ymin':eachObject[2],'xmax': eachObject[3],'ymax':eachObject[4],'width':eachObject[6],'height':eachObject[6]}
        df= df.append(new_row, ignore_index=True)
df.to_csv('/content/result.csv',index = False)
